In [1]:
import pandas as pd
import os
import torchaudio
from tqdm.auto import tqdm
from joblib import Parallel, delayed


PATH_DATASET = "birdclef-2022"

path_csv = os.path.join(PATH_DATASET, "train_metadata.csv")
train_meta = pd.read_csv(path_csv)
def getLength(fn):
    fp = os.path.join(PATH_DATASET, "train_audio", fn)
    metadata = torchaudio.info(fp)
    return metadata.num_frames

train_meta['duration'] = Parallel(n_jobs=os.cpu_count())(delayed(getLength)(fn) for fn in tqdm(train_meta["filename"]))
train_meta['duration'] = train_meta['duration']/32000
train_meta

e:\Anaconda3\envs\audio\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████████████████████████████████████████████████████████████████████| 14852/14852 [00:12<00:00, 1179.92it/s]


,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename,duration
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg,11.102031
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg,47.020406
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg,53.760000
3,afrsil1,[],"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,Oscar Campbell,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:00,https://www.xeno-canto.org/205893,afrsil1/XC205893.ogg,105.482438
4,afrsil1,[],['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,Ross Gallardy,Creative Commons Attribution-NonCommercial-Sha...,3.0,16:30,https://www.xeno-canto.org/207431,afrsil1/XC207431.ogg,3.395906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14847,zebdov,[],"['adult', 'song']",22.6499,120.2872,Geopelia striata,Zebra Dove,Jerome Chie-Jen Ko,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:36,https://www.xeno-canto.org/629769,zebdov/XC629769.ogg,61.056000
14848,zebdov,[],"['adult', 'sex uncertain', 'song']",1.4168,103.7287,Geopelia striata,Zebra Dove,Kim Chuah Lim,Creative Commons Attribution-NonCommercial-Sha...,4.0,08:00,https://www.xeno-canto.org/642415,zebdov/XC642415.ogg,14.856000
14849,zebdov,[],"['adult', 'call', 'sex uncertain']",1.4347,103.7043,Geopelia striata,Zebra Dove,Kim Chuah Lim,Creative Commons Attribution-NonCommercial-Sha...,4.0,09:00,https://www.xeno-canto.org/665873,zebdov/XC665873.ogg,45.576000
14850,zebdov,[],['song'],3.3508,101.2451,Geopelia striata,Zebra Dove,Jelle Scharringa,Creative Commons Attribution-NonCommercial-Sha...,2.5,11:15,https://www.xeno-canto.org/666194,zebdov/XC666194.ogg,21.551000


In [2]:
originBirdGroup = train_meta[['primary_label','duration']].groupby('primary_label').sum()

# Remove too long audio

In [3]:
display(originBirdGroup.describe())
train_meta_filter1 = train_meta[train_meta['duration']<800]
originBirdGroup1 = train_meta_filter1[['primary_label','duration']].groupby('primary_label').sum()
display(originBirdGroup1.describe())

,duration
count,152.000000
mean,4513.277311
std,7629.676663
min,21.106906
25%,538.359359
50%,1481.220625
75%,5465.997094
max,52129.719875


,duration
count,152.000000
mean,4233.241837
std,6851.015457
min,21.106906
25%,538.359359
50%,1481.220625
75%,5027.776313
max,49495.369875


# Remove low rate data (Only enough duration)

In [4]:
enoughdata = originBirdGroup1[originBirdGroup1['duration']>10000]
enoughdata

,duration
primary_label,
bcnher,15323.260469
brnowl,23325.706094
cangoo,14339.855500
commyn,11366.439062
comsan,16710.672156
comwax,10426.756938
dunlin,13991.452344
gamqua,10612.534281
gnwtea,12101.019125


In [5]:
train_meta_filter2 = train_meta_filter1[(~train_meta_filter1['primary_label'].isin(enoughdata.index)) | (train_meta_filter1['rating']>2.5)]

In [6]:
originBirdGroup2 = train_meta_filter2[['primary_label','duration']].groupby('primary_label').sum()
originBirdGroup2show = originBirdGroup2[originBirdGroup1['duration']>10000]
originBirdGroup2show['old'] = enoughdata['duration']
originBirdGroup2show

e:\Anaconda3\envs\audio\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,duration,old
primary_label,,
bcnher,12804.838406,15323.260469
brnowl,20259.694312,23325.706094
cangoo,11888.992937,14339.855500
commyn,9709.682219,11366.439062
comsan,14150.280594,16710.672156
comwax,9273.411750,10426.756938
dunlin,9203.325656,13991.452344
gamqua,9703.908000,10612.534281
gnwtea,10026.751125,12101.019125


In [7]:
enoughdata = originBirdGroup2[originBirdGroup2['duration']>20000]
enoughdata

,duration
primary_label,
brnowl,20259.694312
houspa,23863.378562
norcar,27473.847406
normoc,46278.979813
skylar,21829.648063
wesmea,20721.548531


In [8]:
train_meta_filter3 = train_meta_filter2[(~train_meta_filter1['primary_label'].isin(enoughdata.index)) | (train_meta_filter1['rating']>3)]
originBirdGroup3 = train_meta_filter3[['primary_label','duration']].groupby('primary_label').sum()
originBirdGroup3show = originBirdGroup3[originBirdGroup1['duration']>20000]
originBirdGroup3show['old'] = enoughdata['duration']
originBirdGroup3show

e:\Anaconda3\envs\audio\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
e:\Anaconda3\envs\audio\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,duration,old
primary_label,,
brnowl,17676.733688,20259.694312
houspa,17735.093594,23863.378562
norcar,25191.106375,27473.847406
normoc,43030.234000,46278.979813
skylar,18134.051437,21829.648063
wesmea,19448.163219,20721.548531


In [9]:
originBirdGroup3.describe()

,duration
count,152.000000
mean,3805.976256
std,5600.609917
min,21.106906
25%,538.359359
50%,1481.220625
75%,5027.776313
max,43030.234000


# Make different bird balance

In [10]:
originBirdGroup3[originBirdGroup3['duration']>20000]

,duration
primary_label,
norcar,25191.106375
normoc,43030.234000


In [11]:
import random

train_meta_filter4 = train_meta_filter3.copy()
deleteData = originBirdGroup3[originBirdGroup3['duration']>20000]-20000
for index,row in deleteData.iterrows():
    temp = 0
    deleteIndex = train_meta_filter4[train_meta_filter4['primary_label']==row.name].index
    tempTop = row.duration
    while(temp<tempTop):
        dI = random.randint(0,len(deleteIndex)-1)
        temp =temp + train_meta_filter4[train_meta_filter4.index==deleteIndex[dI]].duration.item()
        train_meta_filter4.drop(deleteIndex[dI],axis=0,inplace=True)
        deleteIndex = deleteIndex.delete(dI)

In [12]:
groupDuration4= train_meta_filter4[['primary_label','duration']].groupby('primary_label').sum()
groupDuration4.describe()

,duration
count,152.000000
mean,3619.141413
std,4642.150450
min,21.106906
25%,538.359359
50%,1481.220625
75%,5027.776313
max,19967.483344


# slice data

In [13]:
train_meta_filter4['segmentNum'] = (train_meta_filter4['duration']//5) + (train_meta_filter4['duration']%5 > 3)

In [14]:
train_meta_filter4['segmentNum'].sum()

108718.0

In [15]:
groupDuration5 = train_meta_filter4[['primary_label','segmentNum']].groupby('primary_label').sum()
groupDuration5.describe()

,segmentNum
count,152.000000
mean,715.250000
std,919.438111
min,4.000000
25%,105.750000
50%,292.500000
75%,989.000000
max,3970.000000


In [16]:
import torch
import math
import os
import noisereduce as nr

@torch.no_grad()
def slice_data(
    data_path: str,
    birdName:str,
    fileName:str,
    reduce_noise: bool = False,
    frame_size: int = 5,
    frame_step: int = 5,
    n_jobs: int = 1,
    channel: int = 0,
    device = "cpu",
    batch_size=5,
):
    pathExport = os.path.join('./', "Slice_data", birdName)
    os.makedirs(pathExport, exist_ok=True)
    fname = os.path.join(data_path, "train_audio", fileName)
    
    waveform, sample_rate = torchaudio.load(fname)
       
    # Change singal to mono
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, axis=0, keepdim=True)
    # Reduce noise
    if reduce_noise:
        waveform = torch.tensor(nr.reduce_noise(
            y=waveform,
            sr=sample_rate,
            use_tqdm=True,
            n_jobs=n_jobs,
        ))
        
    step = int(frame_step * sample_rate)
    size = int(frame_size * sample_rate)
    lenSamples = waveform.size()[-1]
    
    if lenSamples < sample_rate*frame_size:
        waveform = torch.nn.functional.pad(waveform, (0,sample_rate*frame_size-lenSamples), mode='constant', value=0.0)
    else:
        tail_len = (lenSamples-size)%step
        if tail_len > step*0.6:
            waveform = torch.nn.functional.pad(waveform, (0,step-tail_len), mode='constant', value=0.0)
        else:
            waveform = waveform[:,0:-tail_len]

    retList = []
    for i in range(math.floor((waveform.size()[-1] - size) / step)+1):
        begin = i * step
        output_path = fileName.replace('.ogg','') + '_' + str(i) + '.ogg'
        retList.append((fileName,i))
#         print(output_path)
        frame = waveform[:,begin:begin + size]
        torchaudio.save("Slice_data/" + output_path, frame, sample_rate)
        
    return retList
slice_data(PATH_DATASET,train_meta_filter4['primary_label'][0],train_meta_filter4['filename'][0],reduce_noise=True,frame_step=5)

[('afrsil1/XC125458.ogg', 0), ('afrsil1/XC125458.ogg', 1)]

In [17]:
train_meta_final = train_meta_filter4
SegInfo = Parallel(n_jobs=os.cpu_count(),verbose=0)(
    delayed(slice_data)(PATH_DATASET,row.primary_label,row.filename,reduce_noise=True,frame_step=5) for index,row in tqdm(train_meta_final.iterrows(),total=train_meta_final.shape[0]))

100%|██████████████████████████████████████████████████████████████████████████| 13191/13191 [1:09:22<00:00,  3.17it/s]


In [18]:
segList = []
for group in SegInfo:
    for d in group:
        segList.append(d)

In [19]:
train_meta_final_temp = train_meta_final[['primary_label','secondary_labels','type','rating','filename','duration','segmentNum']]
train_meta_final_temp

,primary_label,secondary_labels,type,rating,filename,duration,segmentNum
0,afrsil1,[],"['call', 'flight call']",2.5,afrsil1/XC125458.ogg,11.102031,2.0
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],3.5,afrsil1/XC175522.ogg,47.020406,9.0
2,afrsil1,[],"['call', 'song']",4.0,afrsil1/XC177993.ogg,53.760000,11.0
3,afrsil1,[],"['alarm call', 'call']",4.0,afrsil1/XC205893.ogg,105.482438,21.0
4,afrsil1,[],['flight call'],3.0,afrsil1/XC207431.ogg,3.395906,1.0
...,...,...,...,...,...,...,...
14847,zebdov,[],"['adult', 'song']",2.5,zebdov/XC629769.ogg,61.056000,12.0
14848,zebdov,[],"['adult', 'sex uncertain', 'song']",4.0,zebdov/XC642415.ogg,14.856000,3.0
14849,zebdov,[],"['adult', 'call', 'sex uncertain']",4.0,zebdov/XC665873.ogg,45.576000,9.0
14850,zebdov,[],['song'],2.5,zebdov/XC666194.ogg,21.551000,4.0


In [20]:
Filter_Clip_data = pd.merge(train_meta_final_temp,pd.DataFrame(segList,columns=['filename','seg_index']),how='outer',on='filename')
Filter_Clip_data

,primary_label,secondary_labels,type,rating,filename,duration,segmentNum,seg_index
0,afrsil1,[],"['call', 'flight call']",2.5,afrsil1/XC125458.ogg,11.102031,2.0,0.0
1,afrsil1,[],"['call', 'flight call']",2.5,afrsil1/XC125458.ogg,11.102031,2.0,1.0
2,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],3.5,afrsil1/XC175522.ogg,47.020406,9.0,0.0
3,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],3.5,afrsil1/XC175522.ogg,47.020406,9.0,1.0
4,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],3.5,afrsil1/XC175522.ogg,47.020406,9.0,2.0
...,...,...,...,...,...,...,...,...
109089,zebdov,[],['song'],2.5,zebdov/XC666195.ogg,39.444000,8.0,3.0
109090,zebdov,[],['song'],2.5,zebdov/XC666195.ogg,39.444000,8.0,4.0
109091,zebdov,[],['song'],2.5,zebdov/XC666195.ogg,39.444000,8.0,5.0
109092,zebdov,[],['song'],2.5,zebdov/XC666195.ogg,39.444000,8.0,6.0


In [22]:
Filter_Clip_data.to_csv('Filter_Clip_Data.csv'，index=False)

SyntaxError: invalid character in identifier (3591018716.py, line 1)